In [7]:
# we need to track and account for direction of movement
# assumption is that guard always starts moving northward
# so we can initialize the initial direction as North
# also need to maintain a tracker for visited positions
# we can use a set of tuples (coordinates) to track visited positions
# guard finishes when they exceed a bound of the map
from itertools import cycle
class Map:
    def __init__(self, input_map: list, start_pos: tuple, directions):
        self.map = input_map
        self.start_pos = start_pos
        self.visited = set()
        self.directions = directions

    def countVisited(self):
        return len(self.visited)

    def getNextPosition(self, current_pos: tuple, current_direction: str):
        (row, col) = current_pos
        match current_direction:
            case 'N':
                # move up by 1
                return (row-1, col)
            case 'E':
                # move right by 1
                return (row, col+1)
            case 'S':
                # move down by 1
                return (row+1, col)
            case 'W':
                # move left by 1
                return (row, col-1)

    def checkClear(self, position: tuple):
        # check if position is obstacle
        (row, col) = position
        try:
            if self.map[row][col] == '#':
                return False # update direction in main code
            return True
        except IndexError:
            return True

    def patrol(self):
        # initialize current position and direction
        curr = self.start_pos
        (row, col) = curr
        direction = next(self.directions)
        clear = False

        # loop until guard goes out of bounds
        while 0 <= col <= len(self.map[0])-1 and 0 <= row <= len(self.map)-1:
            # get next location in order
            self.visited.add(curr)
            while not clear:
                next_position = self.getNextPosition(curr, direction)
                clear = self.checkClear(next_position)
                if not clear:
                    # turn 90 deg clockwise
                    direction = next(self.directions)
            # once clear position is finally found, move to new position and mark as visited
            curr = next_position
            (row, col) = curr
            clear = False
        
        # return number of visited cells
        print(self.countVisited())

In [3]:
with open('data/test/6.txt', 'r', encoding='utf-8') as f:
    map_list = [[char for char in line] for line in f.read().splitlines()]
    for row, line in enumerate(map_list):
        for col, char in enumerate(line):
            if char == '^':
                start_idx = (row, col)
    
    map_list[row][col] = '.'
    map1 = Map(map_list, start_idx)
    map1.patrol()


41


In [2]:
with open('data/input/6.txt', 'r', encoding='utf-8') as f:
    map_list = [[char for char in line] for line in f.read().splitlines()]
    for row, line in enumerate(map_list):
        for col, char in enumerate(line):
            if char == '^':
                start_idx = (row, col)
    
    map_list[row][col] = '.'
    map1 = Map(map_list, start_idx)
    map1.patrol()

4967


In [23]:
# part 2: idk LOL just try putting obstruction at every free position of the map and test
# conditions for being in a loop: visit a cell that has already been visited with the same direction
# make unique map object for each variant
# now let's create a subclass of the original Map class that detects loops and also overwrites the patrol function
class Map2(Map):
    def checkLoop(self):
        # initialize current position and direction
        curr = self.start_pos
        (row, col) = curr
        direction = next(self.directions)
        clear = False

        # loop until guard goes out of bounds
        while 0 <= col <= len(self.map[0])-1 and 0 <= row <= len(self.map)-1:
            # get next location in order
            if (curr, direction) in self.visited:
                return True
            self.visited.add((curr, direction))
            while not clear:
                next_position = self.getNextPosition(curr, direction)
                clear = self.checkClear(next_position)
                if not clear:
                    # turn 90 deg clockwise
                    direction = next(self.directions)
            # once clear position is finally found, move to new position and mark as visited
            curr = next_position
            (row, col) = curr
            clear = False
        
        return False

In [24]:
import copy
variants = dict()
with open('data/test/6.txt', 'r', encoding='utf-8') as f:
    original_map = [[char for char in line] for line in f.read().splitlines()]

# print(*original_map, sep='\n')
# get indices of all empty cells 
empty_coords = []
for row, line in enumerate(original_map):
    for col, char in enumerate(line):
        if char == ".":
            empty_coords.append((row, col))
        elif char == '^':
            start_idx = (row, col)

# print(empty_coords)
# make a variant of the map for each entry in empty coord
for (row, col) in empty_coords:
    map_copy = copy.deepcopy(original_map)
    # replace the relevant cell with #
    map_copy[row][col] = "#"
    variants[(row, col)] = map_copy

# print(len(variants))


# check that variants are not carbon copies of one another
# print()
# print(*variants[:5],sep='\n')

loop = 0
for k,v in variants.items():
    directions = cycle(list('NESW'))
    map2 = Map2(v, start_idx, directions)
    # print(k)
    if map2.checkLoop():
        loop += 1
        # print(loop)
print(loop)

6


In [25]:
import copy
from tqdm.notebook import tqdm
variants = dict()
with open('data/input/6.txt', 'r', encoding='utf-8') as f:
    original_map = [[char for char in line] for line in f.read().splitlines()]

# print(*original_map, sep='\n')
# get indices of all empty cells 
empty_coords = []
for row, line in tqdm(enumerate(original_map)):
    for col, char in enumerate(line):
        if char == ".":
            empty_coords.append((row, col))
        elif char == '^':
            start_idx = (row, col)

# print(empty_coords)
# make a variant of the map for each entry in empty coord
for (row, col) in tqdm(empty_coords, desc='Generating variants'):
    map_copy = copy.deepcopy(original_map)
    # replace the relevant cell with #
    map_copy[row][col] = "#"
    variants[(row, col)] = map_copy

# print(len(variants))


0it [00:00, ?it/s]

Generating variants:   0%|          | 0/16088 [00:00<?, ?it/s]

In [ ]:
# check that variants are not carbon copies of one another
# print()
# print(*variants[:5],sep='\n')

loop = 0
for k,v in tqdm(variants.items(), desc='Checking'):
    directions = cycle(list('NESW'))
    map2 = Map2(v, start_idx, directions)
    if map2.checkLoop():
        loop += 1
        # print(loop)
print(loop)

Checking:   0%|          | 0/16088 [00:00<?, ?it/s]

TypeError: Map.__init__() takes 4 positional arguments but 5 were given